In [1]:
import sys, os

import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

src_path = os.path.abspath(os.path.join('../src'))
sys.path.append(src_path)

from wave_analisys import *
from data import RAW_DATA, PROC_DATA, OUTPUT_CHARTS

In [3]:
tests = pd.read_csv(PROC_DATA + '/ultrasound_waves_data.csv')
tests = tests.sort_values(by=['sample_id', 'test_condition', 'wave_type', 'test_id'])
indexes = ['sample_id', 'test_condition', 'wave_type', 'test_id']
tests.set_index(indexes, drop=True, inplace=True)

to_array = lambda text: list(map(float, text.replace('[','').replace(']','').split(',')))
tests['sampling'] = tests['sampling'].apply(to_array)

# tests = filter_tests(tests, sample_id='MA01-02', test_condition='dry', wave_type='s')

In [4]:
threshold_factor = 0.5
times_to_pick= 5

time_cols = []
for t in range(times_to_pick):
    time_cols.append('time_%s' % str(t + 1))
    tests[time_cols[t]] = 0

In [5]:
for test in tests.itertuples():
    arrival_times = []
    
    threshold = max((map(abs, test.sampling))) * threshold_factor     
    arrival_times = zero_crossing_threshold(test, threshold, times_to_pick)
    
    [arrival_times.append(0) for i in range(times_to_pick - len(arrival_times))]
    
    tests.at[test.Index, time_cols] = arrival_times

In [6]:
std_factor = 1.5
mean_times = tests[time_cols].groupby(indexes[0:3]).agg(lambda x: mean_without_outliers(x, std_factor))

In [8]:
def ax_callback(test, cs, ax):

    factor = threshold_factor
    threshold = max((map(abs, test.sampling))) * factor
    ax.axhline(y=threshold, c='red', ls='--', label='Limiar: $\pm$%.2f' % threshold)
    ax.axhline(y=-threshold, c='red', ls='--')
        
    for i, col in enumerate(time_cols):
        time = getattr(test, col)
        if (time > 0):
            ax.plot(time, cs(time), 'o', c='red', ms=3, label='$t_%i$: %.2f$\mu$s' % (i + 1, time), zorder=30)

        mean_time = mean_times.loc[test.Index[0:3]][col]
        if mean_time > 0:
            ax.axvline(x=mean_time, c='green', label='Média %i: %.2f$\mu$s' % (i + 1, mean_time), zorder=10)
            
        handles, labels = ax.get_legend_handles_labels()
        labels.insert(1, '')
        handles.insert(1, mpatches.Patch(color='white'))
        
        ax.legend(handles=handles, labels=labels, prop={'size': 12}, frameon=True, edgecolor='black', ncol=6, loc=4)
        
def save_fig(index, tests_by_sample):
    #plt.savefig(OUTPUT_CHARTS + '/waves_analisys/%s_%s_%s.png' % index, bbox_inches='tight', dpi=300)
    return

waveform_plot(tests, ax_callback=ax_callback, plot_callback=save_fig)

In [9]:
'''
p_waves = filter_tests(mean_times, wave_type='p').reset_index()
p_waves = p_waves.pivot(index='sample_id', columns='test_condition', values='time_1')
p_waves.columns = ['time_1_p_%s' % col_name for col_name in p_waves.columns.to_list()]

'''

s_waves = filter_tests(mean_times, wave_type='s').reset_index()
s_waves = s_waves.pivot(index='sample_id', columns='test_condition', values=time_cols)
s_waves.columns = ['%s_s_%s' % col_name for col_name in s_waves.columns.to_list()]

# results = pd.concat([p_waves, s_waves], axis=1)
results = s_waves
results = results.reindex(sorted(results.columns), axis=1)
display(results)

results.to_csv(PROC_DATA + '/waves/arrival_times.csv', float_format='%.2f')

,time_1_s_dry,time_1_s_saturated,time_2_s_dry,time_2_s_saturated,time_3_s_dry,time_3_s_saturated,time_4_s_dry,time_4_s_saturated,time_5_s_dry,time_5_s_saturated
sample_id,,,,,,,,,,
CB05-01,92.013174,231.318713,94.384263,257.930767,96.836692,274.794038,128.029770,291.235625,131.365319,308.265656
CB05-02,46.661606,45.218494,48.728002,47.455731,50.764610,51.226944,61.955286,53.348250,70.219154,56.010639
EU05-01,43.773146,52.938348,48.477800,56.069011,53.265459,60.157216,56.882996,62.798028,60.525170,66.131547
EU05-02,75.001521,27.710067,78.298574,33.845103,82.194108,40.430714,85.843639,48.986402,91.032522,60.383433
GB02-01,40.725606,41.181082,42.818209,43.169346,44.714264,45.272721,49.947078,48.231736,51.787364,50.149385
GB02-02,59.972852,40.605801,62.972416,42.628080,65.631967,44.863869,72.020046,54.096029,116.479645,56.437996
GB02-03,45.439606,42.541681,59.272183,44.675359,61.437074,46.845141,64.982804,49.201014,72.447707,53.840599
GB02-04,41.933455,42.469899,44.066559,44.643403,45.946472,46.556347,47.796910,54.333140,53.356283,57.419351
GB02-05,59.141473,60.817945,61.761672,65.547134,64.818472,72.645206,68.146653,77.929343,78.008578,80.478678
